In [13]:
import re
import pandas as pd
import os
import time
from bs4 import BeautifulSoup
from pdf2xml import pdf2xml
from convertTxtToDFFns import parseFullDay
from horseInfoFns import parseHorseInfo
from genInfoFns import parseGenInfo
#from horsePlayground import parseFullDay, parseRace
from timesInfoFns import parseTimeInfo

In [14]:
pdf2xml('./../charts/chartsPdf', './../charts/chartsXml')

Converting  2 files


In [23]:
jack = [1,2,3]
print(len(jack))

3


In [2]:
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.112.txt') as file:
    test1 = file.readlines()
with open('./../charts/chartsTxt/eqbPDFChartPlus - 2020-08-11T010651.148.txt') as file:
    test2 = file.readlines()

In [3]:
jack = parseFullDay(test1)

Fractional Times: 22.09 45.87 Final Time: 52.23

 Final Time: 45.65

 Fractional Times: 22.92 46.57 Final Time: 52.51

 Final Time: 45.23



In [5]:
jack.shape

(21, 50)

In [5]:
line = ' Run-Up: 131 feet Temporary Rail: 30 feet'

In [6]:
def parseRunUp(line):
    fullSearch = re.search(r'Run-Up: ([0-9.]*)', line)
    runUp = fullSearch.group(1)

    return runUp

In [7]:
parseRunUp(line)

'131'